In [19]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

In [20]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [21]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [26]:
editDistance('GCTGATCGATCGTACG', genome)

3

In [27]:
editDistance('GATTTACCAGATTGAG', genome)

2

In [28]:
editDistance('GCGTATGC', 'TATTGGCTATACGGTT')

2

In [29]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences

In [66]:
reads = readFastq('ERR266411_1.for_asm.fastq')
# reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']

In [67]:
def overlap(s, p, min_len=3):
    start = 0
    
    while True:
        start = s.find(p[:min_len], start)
        if start == -1:
            return 0
        
        if p.startswith(s[start:]):
            return len(s) - start
    
        start += 1

In [68]:
from collections import defaultdict
def map_kmers(reads, k):
    kmer = defaultdict(set)    
    for read in reads:        
        for i in range(len(read) - k + 1):
            kmer[read[i:i+k]].add(read)
        
    
    return kmer

In [69]:
kmer_map = map_kmers(reads, 30)
# kmer_map = map_kmers(reads, 5)

In [70]:
from itertools import permutations
def overlaps(reads, k=3):
    olaps = dict()
    for a, b in permutations(reads, 2):
        suffix = a[-k:]
        if b in kmer_map[suffix]:
            olap = overlap(a, b)
            if olap > 0:
                olaps[(a, b)] = olap            
    return olaps  

In [73]:
%%time
laps = overlaps(reads, 30)

CPU times: user 20.9 s, sys: 98.8 ms, total: 21 s
Wall time: 21 s


In [78]:
count = 0
for pair, v in laps.items():
    one, other = pair
    if one != other:
        count += 1
print(count)
print(len(laps))


923626
923626
